In [ ]:
# 寫入需要之套件
import os
import re
import time
import pandas as pd
import urllib.request
import re
from datetime import datetime
from selenium import webdriver
from bs4 import BeautifulSoup as Soup
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.support.wait import WebDriverWait
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support import expected_conditions as EC

In [ ]:
# Set up Driver
driver = webdriver.Chrome()

# direct to the URL
url='https://www.google.com.tw/maps'
driver.get(url)
time.sleep(2)

# send search keywords
'''
手動更改之資料:
Queens, New York Tourism/ Brooklyn Tourism/ The Bronx Tourism/ Manhattan Tourism/ Staten Island Tourism/
Queens, New York Restaurant/ Brooklyn Restaurant/ The Bronx Restaurant/ Manhattan Restaurant/ Staten Island Restaurant
'''
keys = "Queens, New York Tourism" # 先手動輸入五大區+Tourism,再輸入五大區+Restuarant(需手動更改之部分,此以Queens, New York為例)
driver.find_element(By.XPATH, '//input[@autofocus="autofocus"]').send_keys(keys)
time.sleep(3)
driver.find_element(By.XPATH, '//button[@aria-label="搜尋"]').click()
time.sleep(3)

In [ ]:
SCROLL_PAUSE_TIME = 5

# 把Scroll下滑到底
# 限制:有時網頁加載會太慢,所以需在每次Scroll下滑之前手動將其拉到頂部
last_height = driver.execute_script("return document.body.scrollHeight")
number = 0
lst_links = []
while True:
    number = number+1
    # Scroll down to bottom
    ele = driver.find_element(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
    driver.execute_script('arguments[0].scrollBy(0, 100000000);', ele)
    # Wait to load page
    time.sleep(SCROLL_PAUSE_TIME)
    # Calculate new scroll height and compare with last scroll height
    print(f'last height: {last_height}')
    ele = driver.find_element(By.XPATH,'//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[2]/div[1]')
    new_height = driver.execute_script("return arguments[0].scrollHeight", ele)
    print(f'new height: {new_height}')
    if number == 100:
        break
    if new_height == last_height:
        break
    print('cont')
    last_height = new_height

# 爬取Google Map搜尋結果頁面之所有結果之連結,存入lst_links中
# 條件:只爬取連結中含有"TW&rclk=1"的連結
list_links = driver.find_elements(By.CLASS_NAME, 'hfpxzc')
for i in range(0, len(list_links)):
    possible_link = list_links[i].get_attribute('href')
    lst_links.append(possible_link)

for i in range(0, len(lst_links)):
    if lst_links[i] == None:
        lst_links[i] = "0"
    if lst_links[i][-9:] != "TW&rclk=1":
        lst_links[i] = "0"

while "0" in lst_links:
    lst_links.remove("0")

In [ ]:
# 網頁爬蟲:若要爬取之資料為五大區+Tourism則使用這段程式碼
# 前往lst_links中所有連結之網頁
location_list = []
cnt = 0
for list in lst_links[0:50]:
    location_list.append([])
    url = list
    browser = webdriver.Chrome()
    browser.get(url)

    # 將網頁滑至最底,以加載所有網頁內容
    n_scroll = 2
    post_url = []
    for i in range(n_scroll):
        scroll = 'window.scrollTo(0, document.body.scrollHeight);'
        browser.execute_script(scroll)
        html = browser.page_source
        soup = Soup(html, 'lxml')
        time.sleep(2)

    # 爬取網頁內地點名稱、評分/評比數、營業時間、Post Code、價格等資訊
    location = browser.find_elements(By.CLASS_NAME, "DUwDvf.fontHeadlineLarge")
    for i in location:
        location_list[cnt].append(i.text)
    rating = browser.find_elements(By.CLASS_NAME, "F7nice")
    for i in rating:
        location_list[cnt].append(i.text)
    opening = browser.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[7]/div[5]/div[2]')
    if opening == []:
        location_list[cnt].append('')
    else:
        for i in opening:
            location_list[cnt].append(i.get_attribute("aria-label"))
    GPS = browser.find_elements(By.CLASS_NAME,"Io6YTe.fontBodyMedium ")
    for i in GPS:
        if "NY" in i.text:
            location_list[cnt].append(i.text)
    if len(location_list[cnt]) == 3:
        location_list[cnt].append("")
    price = browser.find_elements(By.CLASS_NAME, "drwWxc")
    if price == []:
        location_list[cnt].append('')
    else:
        for i in price:
            location_list[cnt].append(i.text)
    browser.close()
    cnt += 1

In [ ]:
# 網頁爬蟲:若要爬取之資料為五大區+Restuarant則使用這段程式碼
# 前往lst_links中所有連結之網頁
location_list = []
cnt = 0
for list in lst_links:
    location_list.append([])
    url = list
    browser = webdriver.Chrome()
    browser.get(url)

    # 將網頁滑至最底,以加載所有網頁內容
    n_scroll = 2
    post_url = []
    for i in range(n_scroll):
        scroll = 'window.scrollTo(0, document.body.scrollHeight);'
        browser.execute_script(scroll)
        html = browser.page_source
        soup = Soup(html, 'lxml')
        time.sleep(2)

    # 爬取網頁內地點名稱、評分/評比數、營業時間、價格、Post Code等資訊
    location = browser.find_elements(By.CLASS_NAME, "DUwDvf.fontHeadlineLarge")
    for i in location:
        location_list[cnt].append(i.text)
    rating = browser.find_elements(By.CLASS_NAME, "F7nice")
    for i in rating:
        location_list[cnt].append(i.text)
    opening = browser.find_elements(By.XPATH, '//*[@id="QA0Szd"]/div/div/div[1]/div[2]/div/div[1]/div/div/div[9]/div[4]/div[2]')
    if opening == []:
        location_list[cnt].append('')
    else:
        for i in opening:
            location_list[cnt].append(i.get_attribute("aria-label"))
    Price = browser.find_elements(By.CLASS_NAME, "mgr77e ")
    for i in Price:
        if "$" in i.text:
            location_list[cnt].append(i.text)
    GPS = browser.find_elements(By.CLASS_NAME, "Io6YTe.fontBodyMedium ")
    for i in GPS:
        if "NY" in i.text:
            location_list[cnt].append(i.text)
    if len(location_list[cnt]) == 3:
        location_list[cnt].append("")
    browser.close()
    cnt += 1

In [ ]:
# 輸出結果:若要輸出至excel之結果為五大區+Tourism則使用這段程式碼
import openpyxl
from openpyxl.styles import Font
from openpyxl.workbook import Workbook

# 開啟新的Excel檔案
my_wb = openpyxl.Workbook()
my_sheet = my_wb.active

# Excel上加入欄位名稱
heading1 = my_sheet.cell(row = 1, column = 1)
heading1.value = "Tourist attraction"
heading1.font = Font(bold=True)
heading2 = my_sheet.cell(row = 1, column = 2)
heading2.value = "Rating"
heading2.font = Font(bold=True)
heading3 = my_sheet.cell(row = 1, column = 3)
heading3.value = "Opening time"
heading3.font = Font(bold=True)
heading4 = my_sheet.cell(row = 1, column = 4)
heading4.value = "Post code"
heading4.font = Font(bold=True)
heading5.value = "Price"
heading5.font = Font(bold=True)
heading5 = my_sheet.cell(row = 1, column = 5)

# 將location_list之結果輸出至Excel表中並儲存
for i in range(len(location_list)):
    for j in range(1, len(location_list[i]) + 1):
        if j == 1:
            cell = str(i + 1) + "A"
        elif j == 2:
            cell = str(i + 1) + "B"
        elif j == 3:
            cell = str(i + 1) + "C"
        elif j == 4:
            cell = str(i + 1) + "D"
        else:
            cell = str(i + 1) + "E"
        cell = my_sheet.cell(row = i + 2, column = j)
        cell.value = location_list[i][j - 1]
    my_wb.save("Queens, New York Tourism.xlsx")

# 後續若有欄位跑掉之問題再進行手動調整